# 3 Training a model to recognize sentiment in text

### **Data source**

- testing dataset Instruction: https://rishabhmisra.github.io/publications/
- Downloadable dataset on Kaggle: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection
- In the workspace: `.../TensorFlow_Zero2Hero/TF_NLP_Zero2Hero/Datasets/Sarcasm_Headlines_Dataset.json`

### **Data Structure**

Example: 

```json
{
    "article_link": "https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5", 
    "headline": "former versace store clerk sues over secret 'black code' for minority shoppers", 
    "is_sarcastic": 0
}
```